In [8]:
import pandas as pd
from collections import deque
import numpy as np

In [30]:
df = pd.read_csv('/Users/wouter/Documents/Finance/crypto/MEXC_Tickers.csv', sep=';', header=None)
df.head(10)

,0,1,2,3,4
0,USDT,USDC,ETH,BTC,TUSD
1,BTC,BTC,BTC,NaN,BTC
2,ETH,ETH,NaN,ETH,ETH
3,USDC,NaN,USDC,USDC,NaN
4,NaN,USDT,USDT,USDT,USDT
5,MX,MX,MX,MX,NaN
6,XRP,XRP,XRP,XRP,NaN
7,DOGE,DOGE,NaN,NaN,DOGE
8,BNB,BNB,NaN,NaN,BNB
9,ELF,NaN,ELF,ELF,NaN


In [31]:
class Matrix:
    def __init__(self, df):
        """
        Initialize the Matrix object with a DataFrame.
        
        Parameters:
        - df: A pandas DataFrame.
        """
        # Keep the original DataFrame for reference
        self.df = df
        self.matrix = self._convert_to_matrix(df.copy())
    
    def _convert_to_matrix(self, df):
        df = df.fillna(0) # Replace NaN with -1
        matrix = df.map(lambda x: 1 if isinstance(x, str) else x).values # Replace all strings with 1
        
        return matrix

    def print_matrix(self):
        print(self.matrix)
    
    def get_matrix(self):
        return self.matrix
    
    def get_df(self):
        return self.df
    
    def print_df(self):
        print(self.df)
        
    def get_currency(self, paths, row, col):
        # Get the base currency from the row index
        base_currency = self.df.iloc[row,col]  # Adjust as necessary if row index is not the currency
        
        # Get the quote currency from the column header
        quote_currency = self.df.iloc[0,col]  # Adjust as necessary if column headers are not the currency
        
        return f"{base_currency}/{quote_currency}"

        
    def get_arbitrage_paths(self, start_row, start_col, max_length, conversion=True):
        all_paths = []
        
        # Queue stores (current_position, path, move count, last_move_was_horizontal, path_visited)
        queue = deque([((start_row, start_col), [(start_row, start_col)], 0, False, set([(start_row, start_col)]))]) 
        
        while queue:
            (row, col), path, moves_count, last_move_was_horizontal, path_visited = queue.popleft()
            
            # Debugging print
            
            if moves_count == max_length:
                if col == start_col:
                    all_paths.append(path)
                continue  # Stop exploring further for this path
            
            if last_move_was_horizontal:
                moves = self.get_vertical_moves(row, col)
            else:
                moves = self.get_horizontal_moves(row, col)
            
            for move in moves:
                new_row, new_col = move
                if (new_row, new_col) not in path_visited:
                    new_path = path + [move]
                    new_path_visited = path_visited.copy()
                    new_path_visited.add((new_row, new_col))
                    queue.append(((new_row, new_col), new_path, moves_count + 1, not last_move_was_horizontal, new_path_visited))
        
        if conversion:
            currency_paths = [
            [self.get_currency(all_paths, row, col) for row, col in path]
            for path in all_paths]
            
        else:
            currency_paths = all_paths
        
        return currency_paths
    
    def get_horizontal_moves(self, row, col):
        cols = self.matrix.shape[1]
        current = (row, col)
        moves = []
        for i in range(cols):
            if self.matrix[row][i] == 1 and (row, i) != current:
                moves.append((row, i))
        return moves
    
    def get_vertical_moves(self, row, col):
        rows = self.matrix.shape[0]
        current = (row, col)
        moves = []
        for i in range(rows):
            if self.matrix[i][col] == 1 and (i, col) != current:
                moves.append((i, col))
        return moves
MEXC = Matrix(df)
MEXC.print_matrix()

[[1 1 1 1 1]
 [1 1 1 0 1]
 [1 1 0 1 1]
 [1 0 1 1 0]
 [0 1 1 1 1]
 [1 1 1 1 0]
 [1 1 1 1 0]
 [1 1 0 0 1]
 [1 1 0 0 1]
 [1 0 1 1 0]
 [1 0 1 1 0]
 [1 0 1 1 0]
 [1 0 1 1 0]
 [1 1 0 1 0]
 [1 1 0 1 0]
 [1 1 0 1 0]
 [1 1 0 1 0]
 [1 1 0 1 0]
 [1 1 0 1 0]
 [1 1 0 1 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 0 1 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 0 1 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]

In [32]:

print(MEXC.get_vertical_moves(0, 4))

results = MEXC.get_arbitrage_paths(1, 1, 3, conversion=True)

print("Traversal Results:")
for path in results:
    print(path)

[(1, 4), (2, 4), (4, 4), (7, 4), (8, 4)]
Traversal Results:
['BTC/USDC', 'BTC/USDT', 'USDT/USDT', 'USDC/USDC']
['BTC/USDC', 'BTC/USDT', 'ETH/USDT', 'ETH/USDC']
['BTC/USDC', 'BTC/USDT', 'MX/USDT', 'MX/USDC']
['BTC/USDC', 'BTC/USDT', 'XRP/USDT', 'XRP/USDC']
['BTC/USDC', 'BTC/USDT', 'DOGE/USDT', 'DOGE/USDC']
['BTC/USDC', 'BTC/USDT', 'BNB/USDT', 'BNB/USDC']
['BTC/USDC', 'BTC/USDT', 'TRX/USDT', 'TRX/USDC']
['BTC/USDC', 'BTC/USDT', 'ATOM/USDT', 'ATOM/USDC']
['BTC/USDC', 'BTC/USDT', 'BCH/USDT', 'BCH/USDC']
['BTC/USDC', 'BTC/USDT', 'SOL/USDT', 'SOL/USDC']
['BTC/USDC', 'BTC/USDT', 'LTC/USDT', 'LTC/USDC']
['BTC/USDC', 'BTC/USDT', 'ADA/USDT', 'ADA/USDC']
['BTC/USDC', 'BTC/USDT', 'EOS/USDT', 'EOS/USDC']
['BTC/USDC', 'BTC/USDT', 'UNI/USDT', 'UNI/USDC']
['BTC/USDC', 'BTC/USDT', 'XEN/USDT', 'XEN/USDC']
['BTC/USDC', 'BTC/USDT', 'AVAX/USDT', 'AVAX/USDC']
['BTC/USDC', 'BTC/USDT', 'SHIB/USDT', 'SHIB/USDC']
['BTC/USDC', 'BTC/USDT', 'BTT/USDT', 'BTT/USDC']
['BTC/USDC', 'BTC/USDT', 'APE/USDT', 'APE/USDC']
[